In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [2]:
class Text:
    def __init__(self, title, author, content):
        self.title = title
        self.author = author
        self.content = content
    
    def get_title(self):
        return self.title
    
    def get_author(self):
        return self.author
    
    def get_content(self):
        return self.content

    def to_request_data(self):
        return {
            "title": self.title,
            "author": self.author,
            "check": False,
            "text": self.content,
            "year": None,
            "emotions": None
        }



def get_response(url):
    return requests.get(url)

def get_page_url_list(start, end):
    page_url_list = []
    for i in range(start, end + 1):
        page_url_list.append("http://poemlove.co.kr/bbs/board.php?bo_table=tb01&sop=and&sst=wr_hit&sod=desc&sfl=&stx=&sca=&page=%d" % (i))
    return page_url_list


def get_content_url_list(url):
    response = get_response(url)
    try:
        content_url_list = []
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            title_list = soup.select('#fboardlist > div.list-board > ul > li a')
            for title in title_list:
                href = title.attrs["href"]
                if("http://" in href):
                    content_url_list.append(href)
        return content_url_list
    except:
        print('Error Occured: get_content_url_list()')

def get_content_text(soup):
    title = soup.select_one('body > div.at-body > div > div > div.col-md-9.pull-right.at-col.at-main > div.view-wrap > section > article > div:nth-child(4) > div.view-content')
    return title.text

def get_author_text(soup):
    author = soup.select_one('body > div.at-body > div > div > div.col-md-9.pull-right.at-col.at-main > div.view-wrap > section > article > div:nth-child(3) > div > strong:nth-child(2)')
    return author.text

def get_title_text(soup):
    title = soup.select_one('body > div.at-body > div > div > div.col-md-9.pull-right.at-col.at-main > div.view-wrap > section > article > h1')
    return title.text

def get_content_data(url):
    response = get_response(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = get_title_text(soup)
    author = get_author_text(soup)
    content = get_content_text(soup)
    return Text(title,author,content)



In [3]:
def get_text_list(page_start, page_end):
    page_url_list = get_page_url_list(page_start, page_end)
    print(page_url_list)
    content_url_list = []
    for page_url in page_url_list:
        content_url_list.extend(get_content_url_list(page_url))
        time.sleep(0.1)
    
    text_list = []
    for content_url in content_url_list:
        text_list.append(get_content_data(content_url))
        if(len(text_list) % 10 == 0):
            print("Get " + str(len(text_list)) + " data!") 
        time.sleep(0.2)

    print("Get " + str(len(text_list)) + " data!")    
    return text_list




In [4]:
def filtering_text_list(text_list):
    #타입 강제하는 방법은?
    fitering_text_list = []
    for text in text_list:
        fitering_text_list.append(filtering_text(text))
    return fitering_text_list

def filtering_text(text):
    title = text.get_title()
    author = text.get_author()
    content = text.get_content()
    content = content.replace(u'\xa0',u'')
    content = content.replace(u'\r',u'\n')
    content = content.replace(u'\t',u'')
    content = content.replace(u'\n ',u'\n\n')
    content = content.replace(u' \n ',u'\n')
    content = content.replace(u' \n ',u'\n')
    title = title.replace(u'\t',u'')
    title = title.replace(u'\n',u'')
    return Text(title,author,content)



In [15]:
text_list = get_text_list(4,50)

['http://poemlove.co.kr/bbs/board.php?bo_table=tb01&sop=and&sst=wr_hit&sod=desc&sfl=&stx=&sca=&page=4', 'http://poemlove.co.kr/bbs/board.php?bo_table=tb01&sop=and&sst=wr_hit&sod=desc&sfl=&stx=&sca=&page=5', 'http://poemlove.co.kr/bbs/board.php?bo_table=tb01&sop=and&sst=wr_hit&sod=desc&sfl=&stx=&sca=&page=6', 'http://poemlove.co.kr/bbs/board.php?bo_table=tb01&sop=and&sst=wr_hit&sod=desc&sfl=&stx=&sca=&page=7', 'http://poemlove.co.kr/bbs/board.php?bo_table=tb01&sop=and&sst=wr_hit&sod=desc&sfl=&stx=&sca=&page=8', 'http://poemlove.co.kr/bbs/board.php?bo_table=tb01&sop=and&sst=wr_hit&sod=desc&sfl=&stx=&sca=&page=9', 'http://poemlove.co.kr/bbs/board.php?bo_table=tb01&sop=and&sst=wr_hit&sod=desc&sfl=&stx=&sca=&page=10', 'http://poemlove.co.kr/bbs/board.php?bo_table=tb01&sop=and&sst=wr_hit&sod=desc&sfl=&stx=&sca=&page=11', 'http://poemlove.co.kr/bbs/board.php?bo_table=tb01&sop=and&sst=wr_hit&sod=desc&sfl=&stx=&sca=&page=12', 'http://poemlove.co.kr/bbs/board.php?bo_table=tb01&sop=and&sst=wr_hit

In [ ]:
text_list = filtering_text_list(text_list)

In [23]:
len(text_list)

KeyboardInterrupt: 

1175

In [ ]:
text_list[10].get_title()

'뿌리에게 - 나희덕'

In [19]:
text_list[19].get_author()

'이해인'

In [20]:
text_list[19].get_content()

'\n가을 바람 이 해 인숲과 바다를 흔들다가이제는 내 안에 들어와나를 깨우는 바람꽃이 진 자리마다열매를 키워놓고햇빛과 손잡은눈부신 바람이 있어가을을 사네바람이 싣고 오는쓸쓸함으로나를 길들이면가까운 이들과의눈물겨운 이별도견뎌낼 수 있으리세상에서 할 수 있는사랑과 기도의아름다운 말향기로운 모든 말깊이 접어두고침묵으로 침묵으로나를 내려가게 하는가을 바람이여하늘 길에 떠가는한 조각 구름처럼아무 매인 곳 없이내가 님을 뵈옵도록끝까지 나를 밀어내는바람이 있어나는 홀로 가도외롭지 않네'

In [21]:
num = 0
url = "http://dapi.facerain.club/text/"
for text in text_list:
    num+=1
    requests.post(url, json=text.to_request_data())
    if(num % 10 == 0):
        print("POST " + str(num)+ " Data!")
    time.sleep(0.1)
        

POST 10 Data!
POST 20 Data!
POST 30 Data!
POST 40 Data!
POST 50 Data!
POST 60 Data!
POST 70 Data!
POST 80 Data!
POST 90 Data!
POST 100 Data!
POST 110 Data!
POST 120 Data!
POST 130 Data!
POST 140 Data!
POST 150 Data!
POST 160 Data!
POST 170 Data!
POST 180 Data!
POST 190 Data!
POST 200 Data!
POST 210 Data!
POST 220 Data!
POST 230 Data!
POST 240 Data!
POST 250 Data!
POST 260 Data!
POST 270 Data!
POST 280 Data!
POST 290 Data!
POST 300 Data!
POST 310 Data!
POST 320 Data!
POST 330 Data!
POST 340 Data!
POST 350 Data!
POST 360 Data!
POST 370 Data!
POST 380 Data!
POST 390 Data!
POST 400 Data!
POST 410 Data!
POST 420 Data!
POST 430 Data!
POST 440 Data!
POST 450 Data!
POST 460 Data!
POST 470 Data!
POST 480 Data!
POST 490 Data!
POST 500 Data!
POST 510 Data!
POST 520 Data!
POST 530 Data!
POST 540 Data!
POST 550 Data!
POST 560 Data!
POST 570 Data!
POST 580 Data!
POST 590 Data!
POST 600 Data!
POST 610 Data!
POST 620 Data!
POST 630 Data!
POST 640 Data!
POST 650 Data!
POST 660 Data!
POST 670 Data!
POST